In [5]:
from transformers import  AutoTokenizer, AutoModelForSeq2SeqLM
from src.dataloaders.seq2seq_samplers.seq2seq_samplers_hypothesis_2 import H2Seq2SeqInferencePersonaSampleV1
from src.hyperparameters.causal_modeling_hyperparameters import H2PersonaChatHyperparametersV1
import torch

In [2]:
model_path = "./models/facebook/bart-base-15or5dmk"
device = "cuda"

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

hyperparameters = H2PersonaChatHyperparametersV1(
	model_name="facebook/bart-base",
	model_architecture="seq2seq",
	chat_history_pair_length=7,
	persona_max_length=14, 
	chat_max_length=19,
)

In [6]:
tokenizer.encode("<c_sep>")

[0, 50265, 2]

In [29]:
sample = H2Seq2SeqInferencePersonaSampleV1(
	tokenizer=tokenizer,
	hyperparameters=hyperparameters,
	dataset_sample={
		"persona": [
      		'I like chocolate ice cream.', 
        	"Sometimes I feel lonely.", 
        	# "I like to play video games."
        ],
		"history": [
    	  	"Hi, do you like ice cream?", 
       		'i do like ice cream but i prefer chocolate ice cream',
			"I feel lonely",
    ]
	}
).get_sample()

for key in sample.keys():
    sample[key] = torch.tensor(sample[key]).unsqueeze(0).to(device)
    
# model(**sample)
answer = model.generate(**sample, max_length=20)
tokenizer.batch_decode(answer, skip_special_tokens=True,)

['i feel lonely because i love chocolate ice cream']